In [2]:
import cv2
import numpy as np
import math
import pandas as pd
from PIL import ImageFont, ImageDraw, Image

# ============================================================
# 도면 ROI -> 미터 스케일(격자) 좌표 추출 도구
#
# 기능
# - (1) 캘리브레이션: 치수선 2점(실제 길이), 원점 1점, x축 방향 1점 = 총 4점 클릭
# - (2) ROI 폴리곤: 꼭짓점 클릭(오목 형태 OK), Enter로 완료
# - (3) ROI 내부 GRID_M 간격의 (X,Y)[m] 격자점 생성 후 CSV 저장
# - (4) CSV 저장 직전 Y축 부호 반전(Y = -Y) 옵션 적용 (수학좌표/ENU 느낌)
#
# 조작
# - 휠: 줌(확대/축소)
# - 우클릭 드래그: 팬(이동)
# - 좌클릭: 점 추가
# - Ctrl+Z 또는 z: 마지막 점 취소(Undo)
# - Enter: 단계 완료
# - ESC: 종료
#
# 주의
# - "점이 줌할 때 움직이는" 문제를 해결하기 위해:
#   화면표시 = (원본 crop) -> (윈도우 크기로 리사이즈)
#   좌표변환은 scale을 직접 곱하지 않고 sx/sy(윈도우/뷰포트 비율)로만 처리함.
# ============================================================

# =========================
# 설정
# =========================
IMAGE_PATH = "image.png"           # 도면 이미지 (한글 경로도 OK)
KNOWN_LENGTH_M = 6.210             # 치수선 실제 길이 (m)
GRID_M = 1.0                       # 격자 간격 (m)
OUT_CSV = "roi_grid_points_1m.csv" # 출력 CSV

# 줌 설정
SCALE_STEP = 0.10
MIN_SCALE = 0.2
MAX_SCALE = 10.0

# 한글 폰트 설정(Windows)
FONT_PATH_CANDIDATES = [
    r"C:\Windows\Fonts\malgun.ttf",
    r"C:\Windows\Fonts\malgunsl.ttf",
    r"C:\Windows\Fonts\NanumGothic.ttf",
]
FONT_SIZE = 20

# Y축 부호 반전 옵션 (저장 직전 적용)
FLIP_Y_BEFORE_SAVE = True

# =========================
# 한글 경로 안전 로드
# =========================
def imread_unicode(path: str):
    data = np.fromfile(path, dtype=np.uint8)
    return cv2.imdecode(data, cv2.IMREAD_COLOR)

img = imread_unicode(IMAGE_PATH)
if img is None:
    raise FileNotFoundError(f"이미지를 불러올 수 없습니다: {IMAGE_PATH}")

# =========================
# PIL 폰트 로드 (없으면 None)
# =========================
def load_korean_font():
    for p in FONT_PATH_CANDIDATES:
        try:
            return ImageFont.truetype(p, FONT_SIZE)
        except:
            pass
    return None

KOR_FONT = load_korean_font()

def draw_text_pil(bgr_img, x, y, text, font=None, color=(30,30,30)):
    """
    bgr_img: OpenCV BGR 이미지
    x,y: 좌상단
    text: 한글 OK
    color: (B,G,R)
    """
    if font is None:
        # 폰트가 없으면 최소 폴백(영문/기호 위주로만 권장)
        cv2.putText(bgr_img, text, (x, y+18), cv2.FONT_HERSHEY_SIMPLEX, 0.65, color, 2)
        return bgr_img

    rgb = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
    pil_im = Image.fromarray(rgb)
    draw = ImageDraw.Draw(pil_im)
    draw.text((x, y), text, font=font, fill=(color[2], color[1], color[0]))
    return cv2.cvtColor(np.array(pil_im), cv2.COLOR_RGB2BGR)

# ============================================================
# 줌/팬/클릭/Undo 가능한 뷰어
# - 화면표시: 원본 crop -> 윈도우 크기로 resize
# - 좌표변환: scale을 따로 곱하지 않고 sx/sy 기반으로만 변환 (점 튐 방지)
# ============================================================
def collect_points_zoom_pan(window_name, base_img, instruction_lines, required_min_points, draw_closed_poly=False):
    H, W = base_img.shape[:2]

    scale = 1.0
    pts = []

    # 뷰포트(원본 crop의 좌상단 offset)
    offset_x, offset_y = 0, 0

    # 팬 드래그 상태
    dragging = False
    drag_start = (0, 0)
    offset_start = (0, 0)

    # 윈도우 크기(고정)
    win_w, win_h = 1400, 900

    def clamp_offsets():
        """
        scale에 따라 원본에서 잘라올 crop(view) 크기를 결정하고,
        offset이 범위를 벗어나지 않도록 보정
        """
        nonlocal offset_x, offset_y

        view_w = int(win_w / scale)
        view_h = int(win_h / scale)

        view_w = max(50, min(view_w, W))
        view_h = max(50, min(view_h, H))

        offset_x = int(np.clip(offset_x, 0, max(0, W - view_w)))
        offset_y = int(np.clip(offset_y, 0, max(0, H - view_h)))
        return view_w, view_h

    def get_disp():
        """
        (disp, view_w, view_h) 반환
        disp: 윈도우 크기(win_w, win_h)로 resize된 화면
        view_w/h: 원본에서 잘라온 crop 크기
        """
        view_w, view_h = clamp_offsets()
        crop = base_img[offset_y:offset_y+view_h, offset_x:offset_x+view_w].copy()
        disp = cv2.resize(crop, (win_w, win_h), interpolation=cv2.INTER_LINEAR)
        return disp, view_w, view_h

    def img_to_disp(px, py, view_w, view_h):
        """
        원본 좌표 -> 화면 좌표
        """
        sx = win_w / view_w
        sy = win_h / view_h
        return int(round((px - offset_x) * sx)), int(round((py - offset_y) * sy))

    def disp_to_img(x, y, view_w, view_h):
        """
        화면 좌표 -> 원본 좌표
        """
        sx = win_w / view_w
        sy = win_h / view_h
        return int(round(offset_x + (x / sx))), int(round(offset_y + (y / sy)))

    def redraw():
        disp, view_w, view_h = get_disp()

        # 점/선 그리기
        for i, (x, y) in enumerate(pts, start=1):
            cx, cy = img_to_disp(x, y, view_w, view_h)
            if 0 <= cx < win_w and 0 <= cy < win_h:
                cv2.circle(disp, (cx, cy), 5, (0, 0, 255), -1)
                cv2.putText(disp, str(i), (cx + 10, cy - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

        if len(pts) >= 2:
            for i in range(len(pts)-1):
                p1 = img_to_disp(*pts[i], view_w, view_h)
                p2 = img_to_disp(*pts[i+1], view_w, view_h)
                cv2.line(disp, p1, p2, (0,255,0), 2)

        if draw_closed_poly and len(pts) >= 3:
            p1 = img_to_disp(*pts[-1], view_w, view_h)
            p2 = img_to_disp(*pts[0], view_w, view_h)
            cv2.line(disp, p1, p2, (0,255,0), 2)

        # 안내문 박스(반투명)
        overlay = disp.copy()
        pad = 12
        line_h = 28
        box_h = pad*2 + line_h*len(instruction_lines)
        box_w = min(980, win_w - 20)

        cv2.rectangle(overlay, (10, 10), (10 + box_w, 10 + box_h), (255,255,255), -1)
        disp = cv2.addWeighted(overlay, 0.65, disp, 0.35, 0)

        y0 = 10 + pad
        for line in instruction_lines:
            disp = draw_text_pil(disp, 10 + pad, y0, line, font=KOR_FONT, color=(30,30,30))
            y0 += line_h

        # 상태 텍스트(영문)
        status = f"scale={scale:.2f}, offset=({offset_x},{offset_y}), points={len(pts)}"
        cv2.putText(disp, status, (12, win_h - 12), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (30,30,30), 2)

        cv2.imshow(window_name, disp)

    def on_mouse(event, x, y, flags, param):
        nonlocal scale, offset_x, offset_y, dragging, drag_start, offset_start

        # 줌
        if event == cv2.EVENT_MOUSEWHEEL:
            view_w, view_h = clamp_offsets()
            img_before = disp_to_img(x, y, view_w, view_h)  # 줌 중심 고정(마우스가 가리키던 원본 점)

            if flags > 0:
                scale = min(MAX_SCALE, scale + SCALE_STEP)
            else:
                scale = max(MIN_SCALE, scale - SCALE_STEP)

            # 줌 후, 같은 원본점이 같은 화면 위치에 오도록 offset 보정
            view_w2, view_h2 = clamp_offsets()
            sx2 = win_w / view_w2
            sy2 = win_h / view_h2
            offset_x = int(round(img_before[0] - (x / sx2)))
            offset_y = int(round(img_before[1] - (y / sy2)))
            clamp_offsets()
            redraw()

        # 팬 시작
        elif event == cv2.EVENT_RBUTTONDOWN:
            dragging = True
            drag_start = (x, y)
            offset_start = (offset_x, offset_y)

        # 팬 이동
        elif event == cv2.EVENT_MOUSEMOVE and dragging:
            dx = x - drag_start[0]
            dy = y - drag_start[1]

            view_w, view_h = clamp_offsets()
            sx = win_w / view_w
            sy = win_h / view_h

            offset_x = int(round(offset_start[0] - dx / sx))
            offset_y = int(round(offset_start[1] - dy / sy))
            clamp_offsets()
            redraw()

        # 팬 종료
        elif event == cv2.EVENT_RBUTTONUP:
            dragging = False

        # 점 추가
        elif event == cv2.EVENT_LBUTTONDOWN:
            view_w, view_h = clamp_offsets()
            px, py = disp_to_img(x, y, view_w, view_h)
            px = int(np.clip(px, 0, W-1))
            py = int(np.clip(py, 0, H-1))
            pts.append((px, py))
            redraw()

    # 창 생성
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, win_w, win_h)
    cv2.setMouseCallback(window_name, on_mouse)

    # 창 항상 위(가능 환경에서만)
    try:
        cv2.setWindowProperty(window_name, cv2.WND_PROP_TOPMOST, 1)
    except:
        pass

    redraw()

    while True:
        key = cv2.waitKey(20) & 0xFF

        # Enter: 완료
        if key == 13:
            if len(pts) < required_min_points:
                print(f"[경고] 최소 {required_min_points}개 점이 필요합니다. 현재: {len(pts)}")
            else:
                break

        # Undo: Ctrl+Z(26) 또는 z/Z
        if key in [26, ord('z'), ord('Z')]:
            if pts:
                pts.pop()
                redraw()

        # ESC: 종료
        if key == 27:
            cv2.destroyWindow(window_name)
            raise SystemExit("사용자에 의해 종료됨(ESC).")

    cv2.destroyWindow(window_name)
    return pts


# ============================================================
# STEP 1) 캘리브레이션
# ============================================================
print("\n[STEP 1/2] 캘리브레이션(스케일/원점/축) 시작")

calib_instructions = [
    "휠: 확대/축소 | 우클릭드래그: 이동(Pan) | 좌클릭: 점 | Ctrl+Z(또는 z): 취소 | Enter: 완료 | ESC: 종료",
    "캘리브레이션 4점 찍기:",
    "1-2) 치수선 양 끝(실제 길이 = KNOWN_LENGTH_M)",
    "3) 원점(0,0)",
    "4) x축 방향(원점에서 오른쪽 방향 점)"
]

calib_pts = collect_points_zoom_pan("CALIB (zoom+pan)", img, calib_instructions, 4, False)
print("[CALIB] 찍힌 점(픽셀):", calib_pts)

p1 = np.array(calib_pts[0], dtype=float)
p2 = np.array(calib_pts[1], dtype=float)
origin_px = np.array(calib_pts[2], dtype=float)
xdir_px = np.array(calib_pts[3], dtype=float)

d_px = np.linalg.norm(p2 - p1)
m_per_px = KNOWN_LENGTH_M / d_px
print(f"[스케일] d_px={d_px:.3f}px, m_per_px={m_per_px:.8f} m/px")

if not (1e-6 < m_per_px < 1.0):
    print("[경고] m_per_px 값이 비정상적으로 보입니다. 치수선 두 점을 정확히 찍었는지 확인하세요!")

# 회전 보정(원점->xdir이 +X가 되도록)
v = xdir_px - origin_px
theta = math.atan2(v[1], v[0])  # 이미지 좌표(y 아래 +)
c, s = math.cos(-theta), math.sin(-theta)
R = np.array([[c, -s],
              [s,  c]], dtype=float)

def px_to_meter(pt_xy_px):
    """
    pt_xy_px: (x,y) pixel
    return: (X,Y) meter (origin 기준, xdir 정렬)
    """
    p = np.array(pt_xy_px, dtype=float)
    dp = p - origin_px
    dp_rot = R @ dp
    xy_m = dp_rot * m_per_px
    return float(xy_m[0]), float(xy_m[1])

def meter_to_px(pt_xy_m):
    """
    (X,Y)m -> pixel(x,y)
    """
    xy_m = np.array(pt_xy_m, dtype=float)
    dp_rot = xy_m / m_per_px
    dp = (R.T @ dp_rot)
    p = dp + origin_px
    return float(p[0]), float(p[1])


# ============================================================
# STEP 2) ROI 폴리곤
# ============================================================
print("\n[STEP 2/2] ROI 폴리곤 지정 시작")

roi_instructions = [
    "휠: 확대/축소 | 우클릭드래그: 이동(Pan) | 좌클릭: 꼭짓점 | Ctrl+Z(또는 z): 취소 | Enter: 완료 | ESC: 종료",
    "ROI 폴리곤 꼭짓점을 여러 번 찍고 Enter로 완료하세요 (최소 3점)"
]

roi_pts = collect_points_zoom_pan("ROI (zoom+pan)", img, roi_instructions, 3, True)
print("[ROI] 찍힌 점(픽셀):", roi_pts)

roi_poly_px = np.array(roi_pts, dtype=np.int32)

# ROI 미리보기 저장
preview = img.copy()
cv2.polylines(preview, [roi_poly_px], True, (0, 255, 0), 3)
cv2.imwrite("roi_polygon_preview.png", preview)
print("[저장] roi_polygon_preview.png")

# ============================================================
# ROI 내부 격자 생성 (미터 단위)
# ============================================================
roi_poly_m = np.array([px_to_meter(pt) for pt in roi_pts], dtype=float)
Xs = roi_poly_m[:, 0]
Ys = roi_poly_m[:, 1]

x_min, x_max = Xs.min(), Xs.max()
y_min, y_max = Ys.min(), Ys.max()

x_vals = np.arange(np.floor(x_min/GRID_M)*GRID_M, np.ceil(x_max/GRID_M)*GRID_M + 1e-9, GRID_M)
y_vals = np.arange(np.floor(y_min/GRID_M)*GRID_M, np.ceil(y_max/GRID_M)*GRID_M + 1e-9, GRID_M)

grid_points = []
for X in x_vals:
    for Y in y_vals:
        x_px, y_px = meter_to_px((X, Y))
        inside = cv2.pointPolygonTest(roi_poly_px, (x_px, y_px), False)
        if inside >= 0:
            grid_points.append((X, Y))

grid_points = np.array(grid_points, dtype=float)

# grid_points가 비어있으면(ROI가 너무 작거나 좌표계가 뒤집혔거나) 에러 방지
if grid_points.size == 0:
    print("[경고] ROI 내부에 격자점이 0개입니다.")
    print(" - ROI 폴리곤을 더 크게 찍었는지")
    print(" - 캘리브레이션 x축 방향/원점이 맞는지")
    print(" - GRID_M 값이 너무 큰지")
    # 그래도 빈 CSV는 만들지 않고 종료
    raise RuntimeError("ROI 내부 격자점이 0개라서 CSV를 생성할 수 없습니다.")

print(f"[결과] ROI 내부 {GRID_M:.1f}m 격자 점 개수: {len(grid_points)}")

# DataFrame 생성
df = pd.DataFrame(grid_points, columns=["X_m", "Y_m"])

# ✅ CSV 저장 직전에 Y축 뒤집기
if FLIP_Y_BEFORE_SAVE:
    df["Y_m"] = -df["Y_m"]
    print("[INFO] Y축 부호를 반전하여 저장합니다. (Y_m = -Y_m)")

# 저장
df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print(f"[저장] {OUT_CSV}")

# ============================================================
# 결과 시각화 저장 (픽셀 프리뷰)
# ============================================================
vis = img.copy()
for (X, Y) in grid_points:
    x_px, y_px = meter_to_px((X, Y))
    cv2.circle(vis, (int(round(x_px)), int(round(y_px))), 2, (255, 0, 0), -1)

cv2.polylines(vis, [roi_poly_px], True, (0, 255, 0), 2)
cv2.imwrite("roi_grid_points_preview.png", vis)
print("[저장] roi_grid_points_preview.png")



[STEP 1/2] 캘리브레이션(스케일/원점/축) 시작
[CALIB] 찍힌 점(픽셀): [(871, 928), (927, 870), (1, 1017), (1504, 1017)]
[스케일] d_px=80.623px, m_per_px=0.07702557 m/px

[STEP 2/2] ROI 폴리곤 지정 시작
[ROI] 찍힌 점(픽셀): [(300, 425), (327, 398), (413, 484), (335, 563), (483, 716), (579, 715), (580, 681), (596, 682), (597, 572), (639, 573), (639, 679), (733, 683), (773, 646), (786, 658), (805, 641), (822, 657), (937, 539), (958, 559), (1232, 285), (1260, 313), (817, 757), (693, 764), (693, 889), (502, 889), (503, 790), (451, 791), (256, 592), (358, 481)]
[저장] roi_polygon_preview.png
[결과] ROI 내부 1.0m 격자 점 개수: 671
[INFO] Y축 부호를 반전하여 저장합니다. (Y_m = -Y_m)
[저장] roi_grid_points_1m.csv
[저장] roi_grid_points_preview.png
